# Exploratory Data Analysis (EDA)

Utilizando el dataset de Olist, una plataforma de comercio electrónico brasileña, se plantea la hipótesis de que es posible predecir qué clientes presentan un patrón de comportamiento asociado al abandono del servicio (churn).

Esta métrica es crucial para las empresas, ya que permite identificar de forma anticipada a los clientes en riesgo y aplicar estrategias de retención proactivas, lo cual puede traducirse en una mejora directa en los ingresos y en la fidelización a largo plazo.

El conjunto de datos abarca compras realizadas entre septiembre de 2016 y octubre de 2018, e incluye información detallada sobre los pedidos, pagos, entregas, localización del cliente y puntuaciones de reseñas.

Para efectos de este análisis, se define churn como aquellos clientes que no han realizado ninguna compra en los últimos 6 meses del periodo registrado. Esta ventana temporal es razonable dado el tipo de productos vendidos en la plataforma, que no exigen compras frecuentes ni suscripciones.

### Importado de librerias y dataset

In [1]:
import sys
print(sys.executable)

/Users/carlos/miniconda3/envs/DS-IBM/bin/python


In [2]:
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import folium

In [6]:
BASE_DIR = Path.cwd().parent
DATA_DIR = (BASE_DIR / "data").resolve()

df = pd.read_csv(DATA_DIR/"processed/processed_dataset.csv")

### Observación general

Lo primero es realizar una observación general del dataset

In [7]:
df.head()


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,total_payment,payment_type,...,worst_review,mean_review_score,last_review,review_comment_message,review_comment_title,total_price,item_count,total_freight_value,customer_city,customer_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,38.71,voucher,...,4,4.0,4,"Não testei o produto ainda, mas ele veio corre...",NaN,29.99,1,8.72,sao paulo,SP
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,141.46,boleto,...,4,4.0,4,Muito bom o produto.,Muito boa a loja,118.70,1,22.76,barreiras,BA
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,179.12,credit_card,...,5,5.0,5,NaN,NaN,159.90,1,19.22,vianopolis,GO
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,72.20,credit_card,...,5,5.0,5,O produto foi exatamente o que eu esperava e e...,NaN,45.00,1,27.20,sao goncalo do amarante,RN
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,28.62,credit_card,...,5,5.0,5,NaN,NaN,19.90,1,8.72,santo andre,SP
